<a href="https://colab.research.google.com/github/Qm1ne/GameDesignBalance/blob/main/chkoba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import random
import numpy as np
import pandas as pd
import json


def create_deck():
    suits = ['♠', '♥', '♦', '♣']
    ranks = list(range(1, 8)) + ['J', 'Q', 'K']
    return [f"{r}{s}" for s in suits for r in ranks]


def card_value(card):
    r = card[:-1]
    if r == 'J': return 8
    if r == 'Q': return 9
    if r == 'K': return 10
    return int(r)


class Player:
    def __init__(self, name, ability=None):
        self.name = name
        self.ability = ability
        self.reset()

    def reset(self):
        self.hand = []
        self.captured = []
        self.points = 0
        self.ability_used = False
        self.ability_was_triggered = False
        self.table_clears = 0

    def apply_ability(self, opponent, table, log, turn, game_id):
        if not self.ability or self.ability_used:
            return

        if self.ability == 'swap_opponent':
            opp_target_card = next((c for c in opponent.hand if card_value(c) == 7 or c.endswith('♦')), None)
            i_have_target = any(card_value(c) == 7 or c.endswith('♦') for c in self.hand)
            my_bad_card = next((c for c in self.hand if card_value(c) < 5 and not c.endswith('♦')), None)

            if opp_target_card and not i_have_target and my_bad_card:
                opponent.hand.remove(opp_target_card)
                self.hand.remove(my_bad_card)
                opponent.hand.append(my_bad_card)
                self.hand.append(opp_target_card)
                log.append({
                    'game_id': game_id,
                    'turn': turn,
                    'player': self.name,
                    'action': f'swap_opponent: {my_bad_card} ↔ {opp_target_card}',
                    'table': json.dumps(table),
                    'hand': json.dumps(self.hand)
                })
                self.ability_used = True
                self.ability_was_triggered = True

        elif self.ability == 'swap_table':
            target_table_card = next((c for c in table if card_value(c) == 7 or c.endswith('♦')), None)
            my_bad_card = next((c for c in self.hand if card_value(c) < 5 and not c.endswith('♦')), None)

            if target_table_card and my_bad_card:
                table.remove(target_table_card)
                self.hand.remove(my_bad_card)
                table.append(my_bad_card)
                self.hand.append(target_table_card)
                log.append({
                    'game_id': game_id,
                    'turn': turn,
                    'player': self.name,
                    'action': f'swap_table: {my_bad_card} ↔ {target_table_card}',
                    'table': json.dumps(table),
                    'hand': json.dumps(self.hand)
                })
                self.ability_used = True
                self.ability_was_triggered = True


    def play_turn(self, table, log, turn, game_id):
        for card in self.hand:
            cv = card_value(card)
            for t in table:
                if card_value(t) == cv:
                    self.hand.remove(card)
                    table.remove(t)
                    self.captured.extend([card, t])
                    if not table:
                        self.table_clears += 1
                    log.append({
                        'game_id': game_id,
                        'turn': turn,
                        'player': self.name,
                        'action': f'capture {card}',
                        'table': json.dumps(table),
                        'hand': json.dumps(self.hand)
                    })
                    return
        card = self.hand.pop(0)
        table.append(card)
        log.append({
            'game_id': game_id,
            'turn': turn,
            'player': self.name,
            'action': f'place {card}',
            'table': json.dumps(table),
            'hand': json.dumps(self.hand)
        })


def compute_final_scores(p1, p2):
    p1.points = 0
    p2.points = 0

    p1.points += p1.table_clears
    p2.points += p2.table_clears

    if len(p1.captured) > len(p2.captured):
        p1.points += 1
    elif len(p2.captured) > len(p1.captured):
        p2.points += 1

    p1_7 = sum(1 for c in p1.captured if card_value(c) == 7)
    p2_7 = sum(1 for c in p2.captured if card_value(c) == 7)
    if p1_7 > p2_7:
        p1.points += 1
    elif p2_7 > p1_7:
        p2.points += 1
    else:
        p1_6 = sum(1 for c in p1.captured if card_value(c) == 6)
        p2_6 = sum(1 for c in p2.captured if card_value(c) == 6)
        if p1_6 > p2_6:
            p1.points += 1
        elif p2_6 > p1_6:
            p2.points += 1


def play_game(p1, p2, game_id):
    deck = create_deck()
    random.shuffle(deck)
    table = deck[:4]
    idx = 4
    log = []
    turn = 0
    p1.reset()
    p2.reset()

    while idx < len(deck) or p1.hand or p2.hand:
        if not p1.hand:
            p1.hand = deck[idx:idx+3]
            idx += 3
        if not p2.hand:
            p2.hand = deck[idx:idx+3]
            idx += 3

        turn += 1
        p1.apply_ability(p2, table, log, turn, game_id)
        p1.play_turn(table, log, turn, game_id)

        turn += 1
        p2.apply_ability(p1, table, log, turn, game_id)
        p2.play_turn(table, log, turn, game_id)

    compute_final_scores(p1, p2)

    winner = (
        "Player1" if p1.points > p2.points else
        "Player2" if p2.points > p1.points else
        "draw"
    )

    return winner, p1.points, p2.points, log, p1.ability_was_triggered, p2.ability_was_triggered


def tournament(n_games=50):
    abilities = ['reveal', 'swap_opponent', 'swap_table', 'bonus7']
    results = np.zeros((len(abilities), len(abilities)))

    logs = []
    winners_data = []
    ability_usage = []

    for i, ability1 in enumerate(abilities):
        for j, ability2 in enumerate(abilities):
            for game_num in range(n_games):
                game_id = f"{ability1}_vs_{ability2}_{game_num}"
                p1 = Player("Player1", ability1)
                p2 = Player("Player2", ability2)

                winner, p1_pts, p2_pts, log, p1_ability_used, p2_ability_used = play_game(p1, p2, game_id)

                if winner == "Player1":
                    results[i][j] += 1
                elif winner == "draw":
                    results[i][j] += 0.5

                winners_data.append({
                    'game_id': game_id,
                    'player1_ability': ability1,
                    'player2_ability': ability2,
                    'winner': winner,
                    'p1_points': p1_pts,
                    'p2_points': p2_pts
                })

                for entry in log:
                    logs.append(entry)

                ability_usage.append({
                    'game_id': game_id,
                    'player': "Player1",
                    'ability': ability1,
                    'used': p1_ability_used
                })
                ability_usage.append({
                    'game_id': game_id,
                    'player': "Player2",
                    'ability': ability2,
                    'used': p2_ability_used
                })

    pd.DataFrame(logs).to_csv("match_logs.csv", index=False)
    pd.DataFrame(winners_data).to_csv("winners.csv", index=False)
    pd.DataFrame(ability_usage).to_csv("ability_usage.csv", index=False)

    return pd.DataFrame(results / n_games, index=abilities, columns=abilities)


def generate_enhanced_ability_datasets():
    logs = pd.read_csv("match_logs.csv")
    ability_usage = pd.read_csv("ability_usage.csv")

    abilities = ['reveal', 'swap_opponent', 'swap_table', 'bonus7']

    log_cols = logs.columns.tolist()
    hand_col = 'hand' if 'hand' in log_cols else 'hand_cards'
    table_col = 'table' if 'table' in log_cols else 'table_cards'

    for ability in abilities:
        dataset = []

        relevant_games = ability_usage[
            (ability_usage['ability'] == ability) &
            (ability_usage['used'] == True)
        ]['game_id'].unique()

        for game_id in relevant_games:
            player_rows = ability_usage[
                (ability_usage['game_id'] == game_id) &
                (ability_usage['ability'] == ability) &
                (ability_usage['used'] == True)
            ]

            for _, player_row in player_rows.iterrows():
                player_name = player_row['player']
                game_logs = logs[
                    (logs['game_id'] == game_id) &
                    (logs['player'] == player_name)
                ]

                captured = []  # on suit les cartes déjà capturées par ce joueur
                for _, log_row in game_logs.iterrows():
                    hand = json.loads(log_row[hand_col])
                    table = json.loads(log_row[table_col])
                    action = log_row['action']

                    # met à jour les cartes capturées si action capture
                    if "capture" in action:
                        parts = action.split()
                        if len(parts) >= 2:
                            captured.append(parts[1])

                    value = 0.0

                    # analyse des cartes
                    all_cards = create_deck()
                    seen_cards = set(hand + table + captured)
                    possible_cards = [c for c in all_cards if c not in seen_cards]

                    # compter 7♦ chez l’adversaire
                    if '7♦' in possible_cards and '7♦' in table:
                        value += 1.273
                    else:
                        # sinon, une autre ♦ ?
                        if any(c.endswith('♦') and c in table for c in possible_cards):
                            value += 0.273
                        else:
                            # sinon menace table clear
                            table_values = set(card_value(c) for c in table)
                            remaining_by_value = {v: 0 for v in table_values}
                            for c in possible_cards:
                                v = card_value(c)
                                if v in remaining_by_value:
                                    remaining_by_value[v] += 1

                            total_possible = len(possible_cards)
                            threat_cards = sum(remaining_by_value.values())

                            if total_possible > 0 and len(table) > 0:
                                prob_adversaire_vide = threat_cards / total_possible
                                value += round(1.0 - prob_adversaire_vide, 3)

                    dataset.append({
                        'game_id': game_id,
                        'turn': log_row['turn'],
                        'player': player_name,
                        'hand_cards': json.dumps(hand),
                        'table_cards': json.dumps(table),
                        'impact_score': value
                    })

        df = pd.DataFrame(dataset)
        filename = f"enhanced_dataset_{ability}.csv"
        df.to_csv(filename, index=False)
        print(f"✅ Enhanced dataset for ability '{ability}' saved to {filename}")



if __name__ == "__main__":
    print("🔷 Running tournament …")
    matrix = tournament(50)
    print("\nWin-rate matrix:\n", matrix.round(2))

    print("\n🔷 Generating training datasets …")
    generate_enhanced_ability_datasets()

🔷 Running tournament …

Win-rate matrix:
                reveal  swap_opponent  swap_table  bonus7
reveal           0.16           0.08        0.19    0.21
swap_opponent    0.24           0.17        0.13    0.07
swap_table       0.12           0.19        0.20    0.18
bonus7           0.17           0.26        0.12    0.19

🔷 Generating training datasets …
✅ Enhanced dataset for ability 'reveal' saved to enhanced_dataset_reveal.csv
✅ Enhanced dataset for ability 'swap_opponent' saved to enhanced_dataset_swap_opponent.csv
✅ Enhanced dataset for ability 'swap_table' saved to enhanced_dataset_swap_table.csv
✅ Enhanced dataset for ability 'bonus7' saved to enhanced_dataset_bonus7.csv


In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump

# ————————————————————————————————————————————
# 🔷 Constantes et utilitaires
# ————————————————————————————————————————————

ALL_CARDS = [f"{r}{s}" for s in ['♠', '♥', '♦', '♣']
             for r in list(map(str, range(1,8)))+['J','Q','K']]
CARD_IDX = {c: i for i, c in enumerate(ALL_CARDS)}

def cards_to_vector(cards):
    vec = np.zeros(len(ALL_CARDS))
    for c in cards:
        if c in CARD_IDX:
            vec[CARD_IDX[c]] = 1
    return vec

def encode_row(row):
    hand = json.loads(row['hand'])
    table = json.loads(row['table'])
    hand_vec = cards_to_vector(hand)
    table_vec = cards_to_vector(table)
    return np.concatenate([hand_vec, table_vec])

def extract_label(action_str):
    if action_str.startswith('capture'):
        return 'capture'
    elif action_str.startswith('place'):
        return 'place'
    elif action_str.startswith('ability'):
        return action_str.replace(':', '_')
    else:
        return 'other'

# ————————————————————————————————————————————
# 🔷 Chargement et préparation des données
# ————————————————————————————————————————————

df = pd.read_csv('simulated_games.csv')
df = df.dropna(subset=['hand', 'table', 'action'])

X = np.vstack(df.apply(encode_row, axis=1))
y = df['action'].apply(extract_label)

# ————————————————————————————————————————————
# 🔷 Division jeu train/test
# ————————————————————————————————————————————

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ————————————————————————————————————————————
# 🔷 Entraînement du modèle
# ————————————————————————————————————————————

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# ————————————————————————————————————————————
# 🔷 Évaluation
# ————————————————————————————————————————————

y_pred = clf.predict(X_test)
print("Rapport de classification :")
print(classification_report(y_test, y_pred))
print(f"Accuracy test: {accuracy_score(y_test, y_pred):.3f}")

# ————————————————————————————————————————————
# 🔷 Sauvegarde du modèle
# ————————————————————————————————————————————

dump(clf, 'ability_model.joblib')
print("✅ Modèle sauvegardé sous 'ability_model.joblib'")

In [ ]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.envs import DummyVecEnv
import gym
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd

# ————————————————————————————————————————————
# 🔷 Définition d'un environnement gym simplifié pour Chkoba
# ————————————————————————————————————————————

class ChkobaEnv(gym.Env):
    def __init__(self):
        super(ChkobaEnv, self).__init__()
        # Observation : vecteur main + table
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(len(ALL_CARDS)*2,), dtype=np.float32)
        # Action : jouer une carte de la main (index 0..len(hand)-1)
        self.action_space = gym.spaces.Discrete(10)  # Exemple: max 10 cartes en main

        self.reset()

    def reset(self):
        self.deck = create_deck()
        np.random.shuffle(self.deck)
        self.hand = self.deck[:10]  # main fixe pour simplification
        self.table = self.deck[10:14]
        return self._get_obs()

    def _get_obs(self):
        hand_vec = cards_to_vector(self.hand)
        table_vec = cards_to_vector(self.table)
        return np.concatenate([hand_vec, table_vec])

    def step(self, action):
        # Jouer carte de main[action] sur table ou capturer
        card = self.hand[action] if action < len(self.hand) else None

        reward = 0
        done = False

        # Simplification: si carte correspond à une carte sur table en valeur => capture
        capture_value = None
        if card:
            card_val = card_value(card)
            for t_card in self.table:
                if card_value(t_card) == card_val:
                    self.table.remove(t_card)
                    self.hand.remove(card)
                    reward = 1  # Capture réussie
                    break
            else:
                # Pas de capture, poser carte sur table
                self.hand.remove(card)
                self.table.append(card)
                reward = -0.1  # Petite pénalité pour pas capturer

        # Terminaison condition simplifiée : main vide
        if len(self.hand) == 0:
            done = True

        obs = self._get_obs()
        info = {}
        return obs, reward, done, info

# ————————————————————————————————————————————
# 🔷 Entraînement PPO (Apprentissage par renforcement)
# ————————————————————————————————————————————

def train_rl_agent():
    env = DummyVecEnv([lambda: ChkobaEnv()])
    model = PPO('MlpPolicy', env, verbose=1)
    model.learn(total_timesteps=50000)
    model.save("chkoba_ppo_model")
    print("✅ Agent PPO entraîné et sauvegardé.")
    return model

# ————————————————————————————————————————————
# 🔷 Clustering sur données simulées
# ————————————————————————————————————————————

def cluster_player_behaviors(log_df, n_clusters=3):
    # Extraire features : nombre d'actions, usage des abilities, points, etc.
    features = []
    players = log_df['player'].unique()

    for p in players:
        p_logs = log_df[log_df['player'] == p]
        total_actions = len(p_logs)
        ability_uses = p_logs['action'].str.contains('ability').sum()
        captures = p_logs['action'].str.contains('capture').sum()
        places = p_logs['action'].str.contains('place').sum()
        features.append([ability_uses/total_actions, captures/total_actions, places/total_actions])

    features = np.array(features)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(features)

    for player, cluster in zip(players, clusters):
        print(f"Player: {player}, Cluster: {cluster}")

    # Visualisation
    plt.figure(figsize=(8,6))
    plt.scatter(features[:,0], features[:,1], c=clusters, cmap='viridis')
    plt.xlabel('Ratio usage ability')
    plt.ylabel('Ratio captures')
    plt.title('Clustering des comportements joueurs')
    plt.show()

# ————————————————————————————————————————————
# 🔷 Exemple d'usage
# ————————————————————————————————————————————

if __name__ == "__main__":
    ALL_CARDS = [f"{r}{s}" for s in ['♠', '♥', '♦', '♣']
                 for r in list(map(str, range(1,8)))+['J','Q','K']]

    print("🔷 Entraînement de l'agent RL...")
    model = train_rl_agent()

    print("🔷 Chargement des logs de parties simulées pour clustering...")
    df_logs = pd.read_csv("simulated_games.csv")
    cluster_player_behaviors(df_logs)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ————————————————————————————————————————————
# 🔷 Chargement des logs de parties simulées
# ————————————————————————————————————————————

def load_game_logs(path="simulated_games.csv"):
    df = pd.read_csv(path)
    return df

# ————————————————————————————————————————————
# 🔷 Analyse du taux de victoire par joueur / ability
# ————————————————————————————————————————————

def analyze_win_rates(df_results):
    # df_results doit contenir au moins : 'game_id', 'winner', 'player1_ability', 'player2_ability'
    win_counts = df_results.groupby(['winner']).size()
    print("🏆 Taux de victoires :")
    print(win_counts)
    return win_counts

# ————————————————————————————————————————————
# 🔷 Analyse d'utilisation des abilities
# ————————————————————————————————————————————

def analyze_ability_usage(df_logs):
    ability_uses = df_logs[df_logs['action'].str.contains('ability')]
    usage_count = ability_uses['player'].value_counts()
    plt.figure(figsize=(8,6))
    sns.barplot(x=usage_count.index, y=usage_count.values, palette='viridis')
    plt.title("Nombre d'utilisation des abilities par joueur")
    plt.ylabel("Nombre d'utilisations")
    plt.xlabel("Joueur")
    plt.tight_layout()
    plt.savefig("ability_usage.png")
    plt.show()

# ————————————————————————————————————————————
# 🔷 Efficacité des abilities vs Victoires
# ————————————————————————————————————————————

def correlate_ability_efficiency(df_results, df_logs):
    # Exemple simple: calcule le ratio de victoire quand l'ability a été utilisée
    ability_players = df_logs[df_logs['action'].str.contains('ability')]['player'].unique()
    results_summary = []

    for player in ability_players:
        games_with_ability = df_logs[df_logs['player'] == player]['game_id'].unique()
        wins = df_results[(df_results['winner'] == player) & (df_results['game_id'].isin(games_with_ability))].shape[0]
        total = df_results[df_results['game_id'].isin(games_with_ability)].shape[0]
        win_rate = wins / total if total > 0 else 0
        results_summary.append({'player': player, 'win_rate_when_ability_used': win_rate})

    df_summary = pd.DataFrame(results_summary)
    print("📊 Résumé efficacité abilities vs victoires:")
    print(df_summary)
    return df_summary

# ————————————————————————————————————————————
# 🔷 Graphique des counters (qui bat qui)
# ————————————————————————————————————————————

def plot_counters(df_results):
    # Suppose df_results contient 'player1_ability', 'player2_ability', 'winner'
    counter_matrix = pd.crosstab(df_results['player1_ability'], df_results['player2_ability'],
                                 values=(df_results['winner']=='Player1'), aggfunc='mean').fillna(0)

    plt.figure(figsize=(8,6))
    sns.heatmap(counter_matrix, annot=True, cmap='coolwarm')
    plt.title("Matrice des counters (victoires Player1 par ability)")
    plt.ylabel("Player1 Ability")
    plt.xlabel("Player2 Ability")
    plt.tight_layout()
    plt.savefig("counter_matrix.png")
    plt.show()

# ————————————————————————————————————————————
# 🔷 Main - Analyse complète
# ————————————————————————————————————————————

if __name__ == "__main__":
    # Chargement des logs et résultats
    df_logs = load_game_logs("simulated_games.csv")

    # Exemple de dataframe résultats, à créer ou charger depuis des résultats de match (à adapter)
    # Ici on simule un dataframe minimal pour illustration
    df_results = pd.DataFrame({
        'game_id': df_logs['game_id'].unique(),
        'winner': ['Player1' if i%2==0 else 'Player2' for i in range(len(df_logs['game_id'].unique()))],
        'player1_ability': ['swap_table'] * len(df_logs['game_id'].unique()),
        'player2_ability': ['swap_opponent'] * len(df_logs['game_id'].unique()),
    })

    print("🔍 Analyse des taux de victoire …")
    analyze_win_rates(df_results)

    print("🔍 Analyse de l'utilisation des abilities …")
    analyze_ability_usage(df_logs)

    print("🔍 Corrélation efficacité abilities et victoires …")
    correlate_ability_efficiency(df_results, df_logs)

    print("🔍 Visualisation des counters …")
    plot_counters(df_results)
